In [1]:
suppressMessages(library(tidyverse))
suppressMessages(library(reshape2))
source('../0_support-files/theme_CRP-MISC.R')



## 3. For cfDNA abundance, are there any differences between moderate and severe MISC?

### prep meta data and deconvolution

In [9]:
TOO_df = read.csv("../1_sample-data/STable8_cfdna-samples.csv") %>% 
    filter(Diagnosis == "MIS-C" & timepoint == "acute") %>%
    rename(scaled_cfDNA = cfDNA_concentration) %>%
    mutate(severity = ifelse(severity == 0 | severity == 1,"Mild/ASX",
                             ifelse(severity == 2 | severity == 3,"Moderate/Severe",
                                    ""))) %>%
    mutate(Diagnosis = ifelse(Diagnosis != "Control_Non-inflammatory", 
                              paste0("Acute ",Diagnosis," ",severity),
                                     Diagnosis)) %>%
    rename(Sample_ID = cfdna_sample_id) %>% 
    mutate(expGroup_plotting = Diagnosis)  %>%
    mutate(group = ifelse(severity == 2, "Moderate MIS-C",
                            ifelse(severity == 3,"Severe MIS-C","other")))%>%
    select(all_of(c(ID_VARS,TISS_COLS)))

TOO_df %>% head()

,PTID,Sample_ID,Diagnosis,timepoint,expGroup_plotting,scaled_cfDNA,severity,group,bcell,colon,⋯,macrophage,megakaryocyte,monocyte,neutrophil,nkcell,pancreas,progenitor,skin,spleen,tcell
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Patient40,cfdna_sample_id_27,Acute MIS-C Moderate/Severe,acute,Acute MIS-C Moderate/Severe,4.784,Moderate/Severe,other,0.123451400,0.006461272,⋯,0.00000000,0.2222888,0.06403882,0.32477842,0.059023780,0.01019488,0.003604042,0.00000000,0.00000000,0.00000000
2,Patient43,cfdna_sample_id_28,Acute MIS-C Moderate/Severe,acute,Acute MIS-C Moderate/Severe,6.667,Moderate/Severe,other,0.048358445,0.007300873,⋯,0.11810660,0.1528253,0.05062569,0.06662735,0.074605030,0.00000000,0.082368089,0.01111545,0.07828432,0.03048947
3,Patient45,cfdna_sample_id_30,Acute MIS-C Moderate/Severe,acute,Acute MIS-C Moderate/Severe,5.999,Moderate/Severe,other,0.120922070,0.000000000,⋯,0.04150866,0.1259113,0.00000000,0.00000000,0.008340871,0.00000000,0.058378850,0.00000000,0.23931450,0.00000000
4,Patient46,cfdna_sample_id_31,Acute MIS-C Moderate/Severe,acute,Acute MIS-C Moderate/Severe,3.712,Moderate/Severe,other,0.008259646,0.000000000,⋯,0.08885321,0.2933872,0.03571192,0.15449954,0.000000000,0.01899220,0.012543671,0.00000000,0.04158856,0.01250763
5,Patient47,cfdna_sample_id_32,Acute MIS-C Moderate/Severe,acute,Acute MIS-C Moderate/Severe,0.922,Moderate/Severe,other,0.001285233,0.000000000,⋯,0.00000000,0.6735906,0.00000000,0.01808105,0.012504200,0.00000000,0.034423650,0.00000000,0.00000000,0.00000000
6,Patient48,cfdna_sample_id_33,Acute MIS-C Moderate/Severe,acute,Acute MIS-C Moderate/Severe,32.748,Moderate/Severe,other,0.002391900,0.015936890,⋯,0.00000000,0.1344302,0.00000000,0.01605826,0.000000000,0.02012707,0.000000000,0.00000000,0.00000000,0.00000000


In [10]:
ID_VARS = c("PTID","Sample_ID","Diagnosis","timepoint","expGroup_plotting","scaled_cfDNA","severity")

TISS_COLS = c('bcell','colon','dendritic','eosinophil','erythroblast','heart','hsc','kidney','liver','lung','macrophage','megakaryocyte','monocyte','neutrophil','nkcell','pancreas','progenitor','skin','spleen','tcell')

## Read in MISC data
TOO_df = read.csv("../1_sample-data/STable8_cfdna-samples.csv") %>% 
    filter(Diagnosis == "MIS-C" & timepoint == "acute") %>%
    rename(scaled_cfDNA = cfDNA_concentration) %>%
    mutate(Diagnosis = ifelse(Diagnosis != "Control_Non-inflammatory", 
                              paste0("Acute ",Diagnosis," ",severity),
                                     Diagnosis)) %>%
    rename(Sample_ID = cfdna_sample_id) %>% 
    mutate(expGroup_plotting = Diagnosis)  %>%
    mutate(group = ifelse(severity == 2, "Moderate MIS-C",
                            ifelse(severity == 3,"Severe MIS-C","other")))%>%
    select(all_of(c(ID_VARS,TISS_COLS)))

ID_VARS <- c(ID_VARS,"group")


##------------------------------------
# Calculate differences between groups
mod <- TOO_df %>% filter(severity == 2) %>% pull(scaled_cfDNA)
sev <- TOO_df %>% filter(severity == 3) %>% pull(scaled_cfDNA)

t <- wilcox.test(x = mod, y = sev)
t


	Wilcoxon rank sum exact test

data:  mod and sev
W = 102, p-value = 0.1948
alternative hypothesis: true location shift is not equal to 0


### Plot

In [11]:
##------------------------------------
# Plot

expGroupPalette = c('#008176','#0000a7')
names(expGroupPalette) <- c("Moderate MIS-C","Severe MIS-C")


WIDTH = 1.6
HEIGHT = 1.5

YLIM = 15


pdf(file=paste0("plots/SupPanelA_severity.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

bxplt <- TOO_df %>%
        ggplot() + 
        geom_boxplot(aes(x=group,y=scaled_cfDNA,fill=group),outlier.shape = NA,size = 0.2, color = "black")+ #, color = "black"
        geom_jitter(aes(x=group,y=scaled_cfDNA),width=.2,height=0,size = 0.01)+
#         geom_point(aes(colour = severity,shape = severity,group=plottingName),position = position_jitterdodge(jitter.width=0.1,seed=100,jitter.height=0),size = 0.5,alpha= 0.5)+       # ,pch=21 ,color = "black"
#         annotate("text", x = 1, y = YLIM - 0.01, label = CELLTYPE,size = 2.5,family = "Helvetica")+
        theme_prevail()+
#         scale_color_manual(values=c('1' = "black",
#                                        '2' = "red")) + 
#         scale_shape_manual(values=c('1'= 16,
#                                        '2'= 4))+
        scale_fill_manual(values=expGroupPalette) + 
        theme(panel.grid.minor = element_blank(),
              axis.title = element_blank(),
             title = element_blank()) +
        scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM))  + 
        coord_cartesian(ylim = c(0,YLIM*1.05))

print(bxplt)

dev.off()

ERROR: Error in FUN(X[[i]], ...): object 'group' not found


## 6. Given the severe GI symptoms associated with MISC as well as the finding that the GI tract may act as a reservoir for SARS-CoV-2 resulting in Spike protein antigenemia in MISC, I would be interested in more detail regarding markers of GI injury. Were there any differences between COVID19 and MISC?

### cfDNA

In [4]:
ID_VARS = c("PTID","Sample_ID","Diagnosis","origin","timepoint","expGroup_plotting","scaled_cfDNA","icu","severity")

TISS_COLS = c('bcell','colon','dendritic','eosinophil','erythroblast','heart','hsc','kidney','liver','lung','macrophage','megakaryocyte','monocyte','neutrophil','nkcell','pancreas','progenitor','skin','spleen','tcell')

## Read in MISC data
TOO_df = read.delim("../../1_sample-output/cfdna_master.tsv") %>% 
    filter(Diagnosis %in% c("MIS-C","COVID-19") & timepoint == "acute" & severity >=2) %>%
    select(all_of(c(ID_VARS,TISS_COLS)))

# ID_VARS <- c(ID_VARS,"group")

## Melt and calculate TOO concentration
TOO_df_melt <- melt(TOO_df,id.vars = ID_VARS)
TOO_df_melt$value_abundance = TOO_df_melt$scaled_cfDNA * TOO_df_melt$value
TOO_df_melt$celltype = TOO_df_melt$variable
TOO_df_melt$fraction = TOO_df_melt$value

##------------------------------------
# Plot


CELLTYPE = "colon"

expGroupPalette = c('#21aaf8','#d42069')
names(expGroupPalette) <- c("MIS-C","COVID-19")

WIDTH = 1.6
HEIGHT = 1.5

YLIM = 0.03


misc_frac <- TOO_df_melt %>% filter(celltype == CELLTYPE) %>% filter(Diagnosis == "MIS-C") %>% pull(fraction)
covid_frac <- TOO_df_melt %>% filter(celltype == CELLTYPE) %>% filter(Diagnosis == "COVID-19") %>% pull(fraction)

wilcox.test(x = misc_frac, y = covid_frac)
                

pdf(file=paste0("plots/SupPanelC_",CELLTYPE,".pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

bxplt <- TOO_df_melt %>%
        filter(celltype == CELLTYPE) %>%
        ggplot() + 
        geom_boxplot(aes(x=Diagnosis,y=fraction,fill=Diagnosis),outlier.shape = NA,size = 0.2, color = "black")+ 
        geom_jitter(aes(x=Diagnosis,y=fraction),width=.2,height=0,size = 0.01)+
        theme_prevail()+
        theme(title = element_text(size = 7.5))+
        labs(title = paste0("cfdna\n", gsub("\\."," ",CELLTYPE)))+
        theme(panel.grid.minor = element_blank(),
              axis.title = element_blank(),
             title = element_blank()) +
        scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM))  + 
        coord_cartesian(ylim = c(0,YLIM*1.05))

print(bxplt)

dev.off()

Warning message in wilcox.test.default(x = misc_frac, y = covid_frac):
“cannot compute exact p-value with ties”



	Wilcoxon rank sum test with continuity correction

data:  misc_frac and covid_frac
W = 199, p-value = 0.4756
alternative hypothesis: true location shift is not equal to 0


png 
  2